**Collaborative Failtering x Knowledge-Based Model**

CF
- past user-item interactions (bookings, clicks) to infer preferences
- based on similarities between users (user-user CF) or items (item-item CF)

KBF
- uses explicit domain knowledge or content features of items(hotels) to decide on recommendations

Goal:

I need to have a subission file that ranks hotels belonging to a user search on the likeliness that the property will be booked

In [ ]:
###### DESTINATION-SPECIFIC HOTEL POPULARITY ######

""""Based on a destination, hotels that are frequantly clicked or booked in that destination are probbably relevant for future user search in that destination."""

def compute_destination_popularity(df):
    """
    Computes normalized popularity score for each hotel within a destination.
    Uses weighted booking and click signals.
    """
    pop_df = df.groupby(['srch_destination_id', 'prop_id'])[['booking_bool', 'click_bool']].sum().reset_index()

    # Weighted score: bookings more important than clicks
    pop_df['cf_score'] = 5 * pop_df['booking_bool'] + 1 * pop_df['click_bool']

    # Normalize within each destination (0 to 1)
    pop_df['cf_score'] = pop_df.groupby('srch_destination_id')['cf_score'].transform(
        lambda x: x / (x.max() + 1e-6)
    )

    return pop_df[['srch_destination_id', 'prop_id', 'cf_score']]


# def add_destination_popularity_score(df):
#     """
#     Adds destination-specific popularity CF score to the dataset.
#     """
#     popularity = compute_destination_popularity(df)
#     df = df.merge(popularity, on=['srch_destination_id', 'prop_id'], how='left')
#     df['cf_score'] = df['cf_score'].fillna(0)  # unseen hotels get score 0
#     return df



In [ ]:
import pandas as pd
# Step 1: Load full original training data (5M rows)
df_raw = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/training_set_VU_DM.csv")

# Step 2: Compute CF scores on full data
popularity_scores = compute_destination_popularity(df_raw)

# Step 3: Merge into your feature-engineered dataset
df_feats = pd.read_csv("nika_candidate_feats.csv")  # or however you generate it
df_feats = df_feats.merge(popularity_scores, on=['srch_destination_id', 'prop_id'], how='left')
df_feats['cf_score'] = df_feats['cf_score'].fillna(0)

df_feats.to_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/features_with_cf_score.csv", index=False)
